In [178]:
import numpy as np
import pandas as pd
import json

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
                                
from collections import Counter

In [179]:
p_dir = '/home/zyp/w/d2l/detect_outfits'


In [180]:
epochs = 60
conf_thres_val = 0.05

exp_dir = p_dir + f'/log/bln_epochs{epochs}/{conf_thres_val}-exp'

In [181]:
def verify_pre(gt_file, dt_file):
    gt = json.load(open(gt_file, 'r'))
    dt = json.load(open(dt_file, 'r'))
    gt_imgs = pd.DataFrame(gt['images'])['id'].to_numpy()
    #print(len(gt_imgs), gt_imgs[:10])
    dt_imgs = pd.DataFrame(dt)['image_id'].to_numpy()
    print(len(dt_imgs), len(set(dt_imgs)))
    
    no_detect = []
    for img_id in gt_imgs:
        if not img_id in dt_imgs:
            no_detect.append(img_id)
    
    return no_detect
    
gt_file = f'{p_dir}/data/origin/valid_local.json'
dt_file = f'{exp_dir}/answer.json'
no_detect = verify_pre(gt_file, dt_file)
len(no_detect)

1593 591


22

In [182]:
origin_train_file = p_dir + '/data/origin/train.json'
data = json.load(open(origin_train_file, 'r'))

In [183]:
cats = pd.DataFrame(data["categories"])[['id', 'name']].set_index('id')
cats_dict = cats.squeeze().to_dict()
cats_dict

{87: 'belt', 1034: 'sunglasses', 131: 'boot', 318: 'cowboy_hat', 588: 'jacket'}

In [186]:
dt = pd.DataFrame(json.load(open(dt_file, 'r')))
dt

,image_id,category_id,bbox,score
0,2857187227633385663,1034,"[446.00026355, 73.00002816, 17.9999869, 9.9999...",0.082424
1,2857187227633385663,1034,"[153.99986895, 93.99997440000001, 20.9999961, ...",0.166514
2,2857187227633385663,1034,"[661.00033095, 2.0479999999878373e-05, 20.9999...",0.370207
3,2857187227633385663,1034,"[314.0000295, 72.9999872, 100.99999100000001, ...",0.873189
4,1855409563895400077,1034,"[250.999819, 388.9997312, 278.99974199999997, ...",0.683113
...,...,...,...,...
1588,6689251422544595225,318,"[294.000092, 112.99993599999999, 418.000215999...",0.206803
1589,6689251422544595225,318,"[25.000004800000003, 327.0000128, 26.9999664, ...",0.257942
1590,6689251422544595225,318,"[153.99975, 358.00053760000003, 72.999972, 35....",0.725709
1591,6689251422544595225,1034,"[343.99994000000004, 299.0000128, 258.00005600...",0.844421


In [184]:
cat_ids = [87, 1034, 131, 318, 588]

In [185]:
def eval(gt_file, dt_file, cat_ids):
    print(gt_file)
    print(dt_file)
    if len(json.load(open(dt_file, 'r'))) == 0:
        print("EMPTY! ", dt_file)
        return
    cocoGt = COCO(gt_file)
    cocoDt = cocoGt.loadRes(dt_file)

    # running evaluation
    cocoEval = COCOeval(cocoGt,cocoDt,iouType='bbox')
    cocoEval.params.catIds  = cat_ids
    cocoEval.evaluate()
    cocoEval.accumulate()

    cocoEval.summarize()

gt_file = f'{p_dir}/data/origin/valid_local.json'
dt_file = f'{exp_dir}/answer.json'
eval(gt_file, dt_file, cat_ids)
print('\n\n')

for cid in cat_ids:
    print(f'{cid}:{cats_dict[cid]}') 
    eval(gt_file, dt_file, [cid])
    print('\n\n')



/home/zyp/w/d2l/detect_outfits/data/origin/valid_local.json
/home/zyp/w/d2l/detect_outfits/log/bln_epochs60/0.05-exp/answer.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.21s).
Accumulating evaluation results...
DONE (t=0.05s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.279
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.500
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.274
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.146
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.282
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.327
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.246
 Average Recall 

---
# 图片分类观察

In [187]:
cat_imgs = {}
for c in cat_ids:
    cat_imgs[c] = set()
for d in dt.to_numpy():
    img_id = d[0]
    cat_id = d[1]
    cat_imgs[cat_id].add(img_id)

cat_imgs

{87: {361189392801794812, 15742841918304422417, 18027551165445256055},
 1034: {1051329269795162,
  12322045450093703,
  30111963444120511,
  109127104396560597,
  234303366212454352,
  237594034503790855,
  309151613107896442,
  380548125617260848,
  582210798814430064,
  611995930297011282,
  612732370816370444,
  640205953571587690,
  663160437824981644,
  718461807596553802,
  722253231118222830,
  746054828961565687,
  832439245772227152,
  928841596103688816,
  1137984495341109671,
  1240669949590208752,
  1285321158120244552,
  1303068949300710487,
  1350285758125050859,
  1474788841078032089,
  1505780244926694019,
  1518973667491580861,
  1571967087446787270,
  1626138972757263702,
  1665878299524628872,
  1767322329468272888,
  1781078496580242076,
  1855409563895400077,
  1916992662359411230,
  1966200073333507223,
  1981991348494928073,
  1982338909631482811,
  1985141223068338433,
  2112915898585481552,
  2218177035611309287,
  2256277289317272015,
  2283305493816510452,
  

In [188]:
for cat_id in cat_imgs:
    cat_name = cats_dict[cat_id]
    os.makedirs(f'{exp_dir}/{cat_name}', exist_ok=True)
    print(f'{exp_dir}/{cat_name}')
    
    img_ids = cat_imgs[cat_id]
    for img_id in img_ids:
        img = imgs.loc[img_id]
        img_file = img.file_name
        sh.copy(f'{exp_dir}/{img_file}', f'{exp_dir}/{cat_name}/')

/home/zyp/w/d2l/detect_outfits/log/bln_epochs60/0.05-exp/belt
/home/zyp/w/d2l/detect_outfits/log/bln_epochs60/0.05-exp/sunglasses
/home/zyp/w/d2l/detect_outfits/log/bln_epochs60/0.05-exp/boot
/home/zyp/w/d2l/detect_outfits/log/bln_epochs60/0.05-exp/cowboy_hat
/home/zyp/w/d2l/detect_outfits/log/bln_epochs60/0.05-exp/jacket


In [203]:
for img_id in cat_imgs[87]:
    img = imgs.loc[img_id]
    in_valid = img_id in valid_imgs.index
    print(img_id, img.file_name, in_valid)

361189392801794812 050333d6ce970afc.jpg True
18027551165445256055 fa2eba455a8b3377.jpg True
15742841918304422417 da79cf50c7aa0a11.jpg True


---
# Eval by YOLOv5

In [205]:
!python ~/w/github/yolov5/val.py --weights {p_dir}/log/bln_epochs40/weights/last.pt \
                            --data {p_dir}/data/yolo2/data.yaml --name {p_dir}/log/yolo_evl/ --iou-thres 0.5 \
                            --conf-thres 0.05 --img 640 

val: data=/home/zyp/w/d2l/detect_outfits/data/yolo2/data.yaml, weights=['/home/zyp/w/d2l/detect_outfits/log/bln_epochs40/weights/last.pt'], batch_size=32, imgsz=640, conf_thres=0.05, iou_thres=0.5, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=../../github/yolov5/runs/val, name=/home/zyp/w/d2l/detect_outfits/log/yolo_evl/, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-237-g50b886d Python-3.8.12 torch-1.11.0+cu102 CPU

Fusing layers... 
Model summary: 213 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning '/home/zyp/w/d2l/detect_outfits/data/yolo2/valid/labels.cache' ima
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        613       1114      0.821      0.517      0.565      0.343
                belt        613          6          1          0          0          0
          sunglasses        613      